Install some necessary libraries, nemreader is to read NEM files, pandas numpy are data processing libraries, and matplotlib is for creating plots.

In [ ]:
!pip install nemreader numpy pandas matplotlib

Import the relevant libraries.

In [ ]:
import numpy as np
import pandas as pd
import os
from nemreader import read_nem_file
import datetime
from matplotlib import pyplot
pd.set_option('display.max_rows', 20)

Read in a sample NEM12 file, read data from the General Consumption (E1) channel

In [ ]:
m = read_nem_file('NEM12Sample.zip')
suffix = 'E1'
readings_list = []
#for suffix in channels:
for nmi in m.readings:
    for channel in m.readings[nmi]:
        for reading in m.readings[nmi][suffix]:
            readings_list.append(reading)

Create a pandas dataframe from the data - similar to an Excel table.

(The dates in this NEM file are 2005, so I'll add ~14 years to make them 2019, in order to demonstrate 'merging' with public holiday data below)

In [ ]:
readings_df = pd.DataFrame(readings_list)
readings_df['t_start'] = readings_df['t_start'].apply(lambda x : x + datetime.timedelta(days=(365.25*14)-5.5))
readings_df['t_end'] = readings_df['t_end'].apply(lambda x : x + datetime.timedelta(days=(365.25*14)-5.5))
readings_df['Date'] = readings_df['t_start'].apply(lambda x : x.date())
readings_df

We'll generate a plot of the 'read_value' data just to confirm it looks as expected

In [ ]:
readings_df['read_value'].plot()

'Resample' the half hourly interval data to hourly data.

As we're downsampling, we must specify which of the two half hourly values to retain (or whether to take the average, sum, or some other user defined function etc.).

In this case we'll take the sum for 'read_value', min for start time ('t_start'), and max for end time ('t_end').

Everything else is the same across both half hourly values, so we can take either the min or max.

In [ ]:
hourly_df = readings_df.resample(on='t_start',rule=datetime.timedelta(hours=1))
myfunc = lambda x : sum(x)/2
aggfuncs = dict.fromkeys(readings_df.columns,max)
aggfuncs.update({'read_value' : myfunc, 't_start' : min})
hourly_df = hourly_df.agg(aggfuncs).reset_index(drop=True)
hourly_df

Generate some other useful columns to work with. The hour of the day, and the day of the week (0=Monday, 6=Sunday)

In [ ]:
hourly_df['hour'] = hourly_df['t_start'].apply(lambda x : x.hour)
hourly_df['weekday'] = hourly_df['t_start'].apply(lambda x : x.weekday())
hourly_df

Retrieve public holiday data from the 'data.gov.au' website, which has aggregated data for Australian public holidays.

Add this in to one dataframe, then create a new dataframe (table) with just the Victorian public holidays. (Any state could be used)

In [ ]:
pub_hols_list_of_dfs = []
pub_hols_url_list = ['https://data.gov.au/data/dataset/b1bc6077-dadd-4f61-9f8c-002ab2cdff10/resource/bda4d4f2-7fde-4bfc-8a23-a6eefc8cef80/download/australian_public_holidays_2019.csv',
                     'https://data.gov.au/data/dataset/b1bc6077-dadd-4f61-9f8c-002ab2cdff10/resource/c4163dc4-4f5a-4cae-b787-43ef0fcf8d8b/download/australian_public_holidays_2020.csv']
for url in pub_hols_url_list:
    pub_hols_list_of_dfs.append(pd.read_csv(url,parse_dates = ['Date']))
pub_hols_df = pd.concat(pub_hols_list_of_dfs,sort=False,join='inner').drop(columns=['Information','More Information'])
pub_hols_df['Date'] = pub_hols_df['Date'].apply(lambda x : x.date())

state = 'vic'
pub_hols_single_state = pub_hols_df[pub_hols_df['Jurisdiction']==state]
pub_hols_single_state

Merge the meter data with the public holiday data. Similar to an SQL join.

Create a new column for 'workdays', defined as days that are not public holidays, and where the weekday value is &lt;5 (0-4 = Monday to Friday)

In [ ]:
merged_df = hourly_df.merge(pub_hols_single_state, how='left',left_on='Date', right_on='Date')
merged_df['workday'] = (merged_df['Holiday Name'].isna()) & (merged_df['weekday'] < 5)
merged_df[35:]

Create a pivot table, showing the average values for workdays vs. non-workdays.

In [ ]:
merged_df.pivot_table(index='hour',columns='workday',values='read_value',margins=True,margins_name='Total',aggfunc=np.mean)

Create hourly data for PVSELL in the required format.

From: https://pvsell.zendesk.com/hc/en-us/articles/115001018408-How-do-I-upload-my-own-solar-production-data-file-

A .csv file with dates in the format d/m hh:mm (e.g., 18/2 01:00)

In [ ]:
pv_sell_df = hourly_df[['t_start','read_value']]
pv_sell_df['t_start'] = pv_sell_df['t_start'].apply(lambda x : x.strftime('%d/%m %H:%M'))
pv_sell_df.to_csv('PVSelloutput.csv',header=False,index=False)
pv_sell_df